<a href="https://colab.research.google.com/github/Servat0r/HLT-Project-2023/blob/master/Text_DaVinci_003_SquaD_Augmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Utilities for `LMQG-SquaD` question augmented dataset using `Text-DaVinci-003` model.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd "/content/drive/MyDrive/Colab Notebooks"

/content/drive/MyDrive/Colab Notebooks


In [ ]:
!pip install "transformers[sentencepiece]"
!pip install "transformers[torch]"
!pip install datasets
!pip install evaluate
!pip install openai
!pip install python-dotenv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 55.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 32.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 115.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 88.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 88.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.5/75.5 kB 1.7 MB/s eta 0:00:00


In [ ]:
from transformers import AutoModel, AutoTokenizer, MT5ForConditionalGeneration, TrainingArguments, Trainer, AdamW, DataCollatorWithPadding
from datasets import load_dataset, Dataset, load_from_disk, load_metric
import numpy as np
import evaluate
import torch
import os
from dotenv import load_dotenv

In [ ]:
load_dotenv()

True

In [ ]:
%run utils.ipynb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24932 sha256=69fec1b330c6bd9fb1ab0fa2fbd219c96d2c90e58e0086b62a9e10e0b70ca3f2
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


Updated!


In [ ]:
model_checkpoint='google/mt5-base'

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. If you see this, DO NOT PANIC! This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=True`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:470: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [ ]:
tokenizer.add_special_tokens({'sep_token': '<sep>', 'additional_special_tokens': ['<answer>', '<context>', '<hl>']})

4

In [ ]:
def load_and_prepare_lmqg_squad_dataset_for_augmentation(
    dataset_name='lmqg/qg_squad', train_dataset_name='lmqg_qg_squad_train_augmented',
    eval_dataset_name='lmqg_qg_squad_eval_augmented', test_dataset_name='lmqg_qg_squad_test_augmented',
    shuffle_seed=None, train_select=None, eval_select=None,
):
  dataset_loading_result = get_dataset(dataset_name, train_dataset_name, eval_dataset_name, test_dataset_name)
  local = dataset_loading_result['local']
  if local:
    train_dataset = dataset_loading_result['train']
    validation_dataset = dataset_loading_result['eval']
    test_dataset = dataset_loading_result['test']
  if not local:
    datasets = load_dataset(dataset_name)
    train_dataset = datasets['train'] \
      .remove_columns(['paragraph_question', 'sentence', 'sentence_answer','paragraph', 'paragraph_sentence']) \
      .rename_column('paragraph_answer', 'context')#.add_column('generated_questions')
    validation_dataset = datasets['validation'] \
      .remove_columns(['paragraph_question', 'sentence', 'sentence_answer', 'paragraph', 'paragraph_sentence']) \
      .rename_column('paragraph_answer', 'context')#.add_column('generated_questions')
    test_dataset = datasets['test'] \
      .remove_columns(['paragraph_question', 'sentence', 'sentence_answer', 'paragraph', 'paragraph_sentence']) \
      .rename_column('paragraph_answer', 'context')#.add_column('generated_questions')
    print(f"Train dataset has {len(train_dataset)} items. Validation dataset has {len(validation_dataset)} items.")

    train_dataset.save_to_disk(train_dataset_name)
    validation_dataset.save_to_disk(eval_dataset_name)
    test_dataset.save_to_disk(test_dataset_name)

  if train_select:
    train_dataset = train_dataset.shuffle(seed=shuffle_seed).select(range(train_select))
  if eval_select:
    validation_dataset = validation_dataset.shuffle(seed=shuffle_seed).select(range(eval_select))

  return train_dataset, validation_dataset, test_dataset

In [ ]:
train_dataset, validation_dataset, test_dataset = load_and_prepare_lmqg_squad_dataset_for_augmentation(shuffle_seed=42)

In [ ]:
def get_question_prompt(item, num_examples=10):
  return f"Propose {num_examples} equivalent questions to the given one. You may use information in context. Prefer shorter questions. " \
  f"Prepend each question with <eoq> and append <eoq> at the end of each question. Do not include the text between <hl> in the questions and do not change named entities in the given question." \
  f"\nQuestion: {item['question']}\nAnswer: {item['answer']}\nContext: {item['context']}"

In [ ]:
print(get_question_prompt(train_dataset[0]))

Propose 10 equivalent questions to the given one. You may use information in context. Prefer shorter questions. Prepend each question with <eoq> and append <eoq> at the end of each question. Do not include the text between <hl> in the questions and do not change named entities in the given question.
Question: What is heresy mainly at odds with?
Answer: established beliefs or customs
Context: Heresy is any provocative belief or theory that is strongly at variance with <hl> established beliefs or customs <hl>. A heretic is a proponent of such claims or beliefs. Heresy is distinct from both apostasy, which is the explicit renunciation of one's religion, principles or cause, and blasphemy, which is an impious utterance or action concerning God or sacred things.


In [ ]:
def get_answer_prompt(item, num_examples=10):
  return f"Propose {num_examples} answers to the given question according to the given context. Prefer shorter answers. " \
  f"Prepend each answer with <eoq> and append <eoq> at the end of each answer. Do not change named entities in the given answer." \
  f"\nQuestion: {item['question']}\nAnswer: {item['answer']}\nContext: {item['context']}"

In [ ]:
from tqdm import tqdm
def augment_dataset(data, num_examples=10, max_tokens=200, engine="text-davinci-003", temperature=0.2, get_prompt=get_question_prompt):
  generated_questions = []
  try:
    for item in tqdm(data):
      prompt = get_prompt(item, num_examples=num_examples)
      response = openai.Completion.create(engine=engine, prompt=prompt, max_tokens=max_tokens, temperature=temperature, n=1)
      generated_text = response.choices[0].text.strip()
      questions = generated_text.split('<eoq>')
      questions = [item.strip() for item in questions]
      questions = [item for item in questions if len(item) > 0]
      generated_questions.append(questions)
    print(generated_questions)
    data = data.add_column('generated_questions', generated_questions)
    return data
  except Exception as ex:
    return generated_questions

#### Augmenting first 1000 entries

In [ ]:
import openai
api_key = os.getenv('OPENAPIKEY')
openai.api_key = api_key

In [ ]:
train_data = train_dataset.shuffle(seed=42).select(range(1000))

In [ ]:
new_train_data = augment_dataset(train_data, num_examples=4)

 33%|███▎      | 331/1000 [14:28<29:14,  2.62s/it]


KeyboardInterrupt: ignored

In [ ]:
lengths = np.array([len(item) for item in new_train_data['generated_questions']])

In [ ]:
incorrect_indexes = np.where(lengths != 4)
incorrect_outputs = lengths[lengths != 4]
incorrect_indexes, incorrect_outputs

In [ ]:
for index in incorrect_indexes[0]:
  print(new_train_data['generated_questions'][index])

In [ ]:
generated_questions = list(new_train_data['generated_questions'])

In [ ]:
for index in incorrect_indexes[0]:
  generated_questions[index] = generated_questions[index][1:]

In [ ]:
for index in incorrect_indexes[0]:
  print(generated_questions[index])

In [ ]:
new_train_data = new_train_data.remove_columns(['generated_questions'])

In [ ]:
new_train_data = new_train_data.add_column('generated_questions', column=generated_questions)

In [ ]:
new_train_data

In [ ]:
import json
with open('generated_questions.json', 'w') as fp:
  json.dump(new_train_data['generated_questions'], fp, indent=2)

In [ ]:
new_train_data['generated_questions'][0]

In [ ]:
!pip install bert_score

from evaluate import load
bertscore = load('bertscore')

In [ ]:
import numpy as np
def compute_new_questions_quality(questions, references, which='f1'):
  questions_array = np.zeros((len(questions), len(questions[0])), dtype=object)
  for i in range(len(questions)):
    questions_array[i, :] = questions[i][:]
  results = np.zeros_like(questions_array, dtype=np.float64)
  batch_size, num_questions = results.shape
  for i in range(num_questions):
    predictions = questions_array[:, i]
    results[:, i] = bertscore.compute(predictions=predictions, references=references, lang='en')[which]
  return results

In [ ]:
all_references = [item['question'] for item in new_train_data]

In [ ]:
results = compute_new_questions_quality(new_train_data['generated_questions'], all_references, which='f1')

In [ ]:
np.where(results < 0.8)[0].shape, np.where(results < 0.79)[0].shape

In [ ]:
results_list = []
for result in results:
  results_list.append(list(result))

In [ ]:
with open('f1_scores.json', 'w') as fp:
  json.dump(results_list, fp)

In [ ]:
import json
with open('generated_questions.json', 'r') as fp:
  generated_questions = json.load(fp)

In [ ]:
generated_questions[0]

In [ ]:
train_data[0]

In [ ]:
for new_questions, train_item in zip(generated_questions, train_data):
  new_questions.append(train_item['question'])

In [ ]:
train_data = train_data.add_column('questions', column=generated_questions)

In [ ]:
train_data['questions'][0]

In [ ]:
train_data.save_to_disk('lmqg_squad_augmented_train')

In [ ]:
eval_data = validation_dataset.shuffle(seed=42)
eval_data.save_to_disk('lmqg_squad_augmented_eval')

In [ ]:
test_data = validation_dataset.shuffle(seed=42)
test_data.save_to_disk('lmqg_squad_augmented_test')

#### Another 1000 entries

In [ ]:
import openai
api_key = os.getenv('OPENAPIKEY')
openai.api_key = api_key

In [ ]:
train_data = train_dataset.shuffle(seed=42).select(range(1000, 2000))

In [ ]:
new_train_data = augment_dataset(train_data, num_examples=4)

100%|██████████| 1000/1000 [45:07<00:00,  2.71s/it]

[['What did Netscape attempt to use to produce a competitive browser?', 'What model did Netscape use to create a competitive browser?', 'What did Netscape use to create a competitive browser?', 'What did Netscape use to develop Firefox?'], ['What public transportation ideas is the City of New Haven currently considering?', 'What is the City of New Haven currently considering to replace trolley service?', 'What is the City of New Haven considering to restore public transportation?', 'What is the City of New Haven looking into to replace the last trolley route?'], ['What is a "tag"?', 'What is the purpose of a "tag"?', 'What is the process of a "tag"?', 'What is the significance of a "tag"?'], ['What are centrifugation and pressing examples of?', 'What are prime examples of modern separation techniques?', 'What are two modern separation techniques?', 'What are two examples of modern separation techniques?'], ['How has political upheaval in the countries around Samoa impacted its economy?

Flattening the indices:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
from time import sleep
sleep(180)

In [ ]:
lengths = np.array([len(item) for item in new_train_data['generated_questions']])

In [ ]:
incorrect_indexes = np.where(lengths != 4)
incorrect_outputs = lengths[lengths != 4]
incorrect_indexes, incorrect_outputs

((array([ 41, 188, 229, 503, 557, 600, 611, 742, 974]),),
 array([5, 5, 5, 5, 5, 5, 5, 5, 5]))

In [ ]:
for index in incorrect_indexes[0]:
  print(new_train_data['generated_questions'][index])

['umerated below are 4 equivalent questions to the given one.', "What did Bohr say about the electron of an atom, citing Planck's work?", "What did Bohr state about the electron of an atom, citing Planck's work?", "What did Bohr assert about the electron of an atom, citing Planck's work?", "What did Bohr propose about the electron of an atom, citing Planck's work?"]
['.', "Who named London the world's capital of higher education?", 'Which professional services network called London the global capital of higher education?', 'What report referred to London as the global capital of higher education?', "Which report declared London as the world's capital of higher education?"]
['Who is the current President of Cyprus?', 'Who is the President of Cyprus?', 'Who was elected President of Cyprus in 2013?', 'Who is Nicos Anastasiades?', 'Who succeeded Dimitris Christofias as President of Cyprus?']
['"In the autumn, when the weather becomes cold and tempestuous, they leave us, and fly away to the

In [ ]:
generated_questions = list(new_train_data['generated_questions'])

In [ ]:
for index in incorrect_indexes[0]:
  generated_questions[index] = generated_questions[index][1:]

In [ ]:
for index in incorrect_indexes[0]:
  print(generated_questions[index])

["What did Bohr say about the electron of an atom, citing Planck's work?", "What did Bohr state about the electron of an atom, citing Planck's work?", "What did Bohr assert about the electron of an atom, citing Planck's work?", "What did Bohr propose about the electron of an atom, citing Planck's work?"]
["Who named London the world's capital of higher education?", 'Which professional services network called London the global capital of higher education?', 'What report referred to London as the global capital of higher education?', "Which report declared London as the world's capital of higher education?"]
['Who is the President of Cyprus?', 'Who was elected President of Cyprus in 2013?', 'Who is Nicos Anastasiades?', 'Who succeeded Dimitris Christofias as President of Cyprus?']
['Where were the Swallows flying?', 'What direction were the Swallows flying in?', 'What direction did the Swallows fly?', 'Which way were the Swallows flying?']
['Who did Tito openly support in the Greek Civil

In [ ]:
new_train_data = new_train_data.remove_columns(['generated_questions'])

In [ ]:
new_train_data = new_train_data.add_column('generated_questions', column=generated_questions)

In [ ]:
new_train_data

Dataset({
    features: ['answer', 'question', 'context', 'generated_questions'],
    num_rows: 1000
})

In [ ]:
import json
with open('generated_questions_1000_2000.json', 'w') as fp:
  json.dump(new_train_data['generated_questions'], fp, indent=2)

In [ ]:
new_train_data['generated_questions'][0]

['What did Netscape attempt to use to produce a competitive browser?',
 'What model did Netscape use to create a competitive browser?',
 'What did Netscape use to create a competitive browser?',
 'What did Netscape use to develop Firefox?']

In [ ]:
!pip install bert_score

from evaluate import load
bertscore = load('bertscore')

In [ ]:
import numpy as np
def compute_new_questions_quality(questions, references, which='f1'):
  questions_array = np.zeros((len(questions), len(questions[0])), dtype=object)
  for i in range(len(questions)):
    questions_array[i, :] = questions[i][:]
  results = np.zeros_like(questions_array, dtype=np.float64)
  batch_size, num_questions = results.shape
  for i in tqdm(range(num_questions)):
    predictions = questions_array[:, i]
    results[:, i] = bertscore.compute(predictions=predictions, references=references, lang='en')[which]
  return results

In [ ]:
all_references = [item['question'] for item in new_train_data]

In [ ]:
results = compute_new_questions_quality(new_train_data['generated_questions'], all_references, which='f1')

  0%|          | 0/4 [00:00<?, ?it/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
np.where(results < 0.8)[0].shape, np.where(results < 0.79)[0].shape

((0,), (0,))

In [ ]:
results_list = []
for result in results:
  results_list.append(list(result))

In [ ]:
with open('f1_scores_1000_2000.json', 'w') as fp:
  json.dump(results_list, fp)

In [ ]:
for new_questions, train_item in zip(generated_questions, train_data):
  new_questions.append(train_item['question'])

In [ ]:
new_train_data = train_data.add_column('questions', column=generated_questions)

### Putting the two blocks together

In [ ]:
train_data_1000_2000 = new_train_data

In [ ]:
train_data_0_1000 = load_from_disk('lmqg_squad_augmented_train')

In [ ]:
train_data_0_1000[0], train_data_1000_2000[0]

({'answer': "National Hockey League's Carolina Hurricanes",
  'question': 'What is the only professional sports team in North Carolina?',
  'context': "The <hl> National Hockey League's Carolina Hurricanes <hl> franchise moved to Raleigh in 1997 from Hartford, Connecticut (where it was known as the Hartford Whalers). The team played its first two seasons more than 60 miles away at Greensboro Coliseum while its home arena, Raleigh Entertainment and Sports Arena (later RBC Center and now PNC Arena), was under construction. The Hurricanes are the only major league (NFL, NHL, NBA, MLB) professional sports team in North Carolina to have won a championship, winning the Stanley Cup in 2006, over the Edmonton Oilers. The city played host to the 2011 NHL All-Star Game.",
  'questions': ['What team is the only major league professional sports team in North Carolina?',
   'Which professional sports team in North Carolina has won a championship?',
   'What is the only NHL team in North Carolina?',

In [ ]:
train_data_0_1000['questions'][0]

['What team is the only major league professional sports team in North Carolina?',
 'Which professional sports team in North Carolina has won a championship?',
 'What is the only NHL team in North Carolina?',
 'Which team won the Stanley Cup in 2006?',
 'What is the only professional sports team in North Carolina?']

In [ ]:
from datasets import concatenate_datasets
train_data_0_2000 = concatenate_datasets([train_data_0_1000, train_data_1000_2000])

In [ ]:
train_data_0_2000['questions']

[['What team is the only major league professional sports team in North Carolina?',
  'Which professional sports team in North Carolina has won a championship?',
  'What is the only NHL team in North Carolina?',
  'Which team won the Stanley Cup in 2006?',
  'What is the only professional sports team in North Carolina?'],
 ['What branch of Islam was Gaddafi raised in?',
  'Which type of Islam did Gaddafi practice?',
  'What kind of Islam did Gaddafi follow?',
  "What was Gaddafi's Islamic denomination?",
  'What variety of Islam was Gaddafi raised in?'],
 ['What is the grammatical term for the state of the action of the verb at the time specified by its tense?',
  'What is the distinction between perfective and imperfective verbs?',
  'What is the difference between aspect and tense?',
  'What is the meaning of aspect in Czech verbs?',
  'What describes the state of the action of the verb at the time specified by its tense?'],
 ['Were any Burmese films used as a tool for influencing pe

In [ ]:
r = [len(item) for item in train_data_0_2000['questions']]

In [ ]:
train_data_0_2000.save_to_disk('lmqg_squad_augmented_train_0_2000')

Saving the dataset (0/1 shards):   0%|          | 0/2000 [00:00<?, ? examples/s]